In [4]:
import numpy as np
import cv2
import os

In [5]:
def eve_distortion(eve_intensities): #from the reconstructed image in uint8 type
    feve_intensities = eve_intensities.astype(np.float)
    fintensities = intensities.astype(np.float)
    dif = (fintensities -  feve_intensities)
    eve_dist_mat = dif**2
    occurance_sum = height*width #?!
    eve_dist = eve_dist_mat.sum()/occurance_sum
    return eve_dist

In [6]:
all_pairs1 ={}
all_pairs2 ={}
all_ths = {}
for size in range(3,11):
    cells_num = 2**size
    pairs1_indexes = np.arange(0,2**(size-1),1)
    pairs1 = pairs1_indexes - (2**size-1)

    th = 2**(size-2)-1 # max threshold = 2**(size-1)-1
    pairing_rounds = np.floor(th/2)
    pairs2_indexes = [] # In second half index and value are the same
    for j in range(int(pairing_rounds)):
        n = int(((2**(size)-1)-(2**(size-1))-(2*j+1))/th)
        for i in range(n+1):
            pairs2_indexes = np.append(pairs2_indexes, 2**(size-1)+2*j+i*th)
            pairs2_indexes = np.append(pairs2_indexes, 2**(size-1)+2*j+1+i*th)
        if (2**(size-1)+2*j+(n+1)*th) <= (2**(size)-1):
            pairs2_indexes = np.append(pairs2_indexes, 2**(size-1)+2*j+(n+1)*th)

    if np.mod(th,2):
        n = int((2**(size)-1-2**(size-1)-(th-1))/th)# 128+4+n*th =<255
        for i in range(n+1):
            pairs2_indexes = np.append(pairs2_indexes,2**(size-1)+th-1+i*th)
            
    all_pairs2[size] = pairs2_indexes
    all_pairs1 [size] = pairs1
    all_ths[size] = th

In [7]:
img_file = 'lena_gray_512.tif'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)

# At Eve #
eve_msb0_intensities = np.zeros([height,width])
eve_msb1_intensities = np.zeros([height,width])

for i in range(0,int(height/8)):
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_ac = np.delete(nq_coef, [0,0])
        nq_coef_dc = nq_coef[0,0]
        
        eve_ac_msb0 = nq_coef_ac.copy()
        eve_ac_msb1 = nq_coef_ac.copy()
        
        for m in range(np.size(nq_coef_ac)):
            org_coef = nq_coef_ac[m]
            if org_coef != 0:
                coef_size = np.floor(np.log2(abs(org_coef)))+1
            else:
                coef_size = 0
                
            if coef_size > 2:
                pairs1_in_size = all_pairs1[coef_size]
                pairs2_in_size = all_pairs2[coef_size]

                if org_coef in pairs1_in_size:
                    pos = np.argwhere(pairs1_in_size == org_coef)
                    if np.mod(pos,2):
                        eve_ac_msb0[m] = pairs2_in_size[pos]
                        
                    if not np.mod(pos,2):
                        eve_ac_msb1[m] = pairs2_in_size[pos]

                if org_coef in pairs2_in_size:
                    pos = np.argwhere(pairs2_in_size == org_coef)
                    if not np.mod(pos,2):
                        eve_ac_msb0[m] = pairs1_in_size[pos]
                        
                    if np.mod(pos,2):
                        eve_ac_msb1[m] = pairs1_in_size[pos]
        
        
        eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, nq_coef_dc)
        eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])
        
        eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
        eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
        eve_msb0_blk = np.round(eve_msb0_blk2)  
        eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk
        
        
        eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, nq_coef_dc)
        eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])
        
        eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
        eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
        eve_msb1_blk = np.round(eve_msb1_blk2)  
        eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk

In [8]:
eve_msb0_uint8 =eve_msb0_intensities.astype(np.uint8)
cv2.imshow('eve_msb0', eve_msb0_uint8)

filename = 'eve_msb0_category_ths.png'
cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb0_uint8)
# cv2.imwrite(filename, eve_msb0_uint8)
cv2.waitKey(1000)

eve_msb1_uint8 =eve_msb1_intensities.astype(np.uint8)
cv2.imshow('eve_msb1', eve_msb1_uint8)

filename = 'eve_msb1_category_ths.png'
cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb1_uint8)
# cv2.imwrite(filename, eve_msb1_uint8)
cv2.waitKey(1000)

-1

In [9]:
cv2.destroyAllWindows()

In [10]:
eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_intensities))
eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_intensities))
print(eve_msb0_dist_db)
print(eve_msb1_dist_db)

28.113722579978067
27.97126476305222
